In [5]:
import pandas as pd
import numpy as np
import os
import glob

# Directory where the original files are located
input_directory = '/Users/lutianshu/Desktop/Polymax/Raw/'

# Directory where the cleaned files will be saved
output_directory = '/Users/lutianshu/Desktop/Polymax/Clean/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get a list of all .xlsx files in the input directory
files = glob.glob(input_directory + '*.xlsx')

# Loop over the files
for file_path in files:
    xl1 = pd.ExcelFile(file_path)
    df1 = xl1.parse('ProdRunQualityRecord')

    # Your transformations here
    gl_rows = df1[df1['Production Run Quality Record'].str.contains('GL', na=False)]
    gl_rows['Box Number'] = gl_rows['Production Run Quality Record'].str.split(' ').str[1]
    gl_rows_copy = gl_rows.copy()
    gl_rows_copy['Box Number'] = gl_rows_copy['Box Number'].str.split(' ').str[1]
    
    # Drop columns where all values are NaN
    gl_rows_copy = gl_rows_copy.dropna(axis=1, how='all')

    gl_rows_copy = gl_rows_copy.rename(columns={
        "Unnamed: 2": "Hardness, A",
        "Unnamed: 3": "Viscosity@67022",
        "Unnamed: 4": "Viscosity@11170",
        "Unnamed: 5": "Viscosity@1340",
        "Unnamed: 6": "Viscosity@223.4",
        "Unnamed: 7": "Viscosity@67.07",
        "Unnamed: 8": "Unnamed: 8",
        "Unnamed: 9": "A Value",
        "Unnamed: 10": "B-Value",
        "Unnamed: 12": "Tensile Strength MPa",
        "Unnamed: 13": "Elongation",
        "Unnamed: 14": "50% Modulus",
        "Unnamed: 15": "100% Modulus",
        "Unnamed: 16": "200% Modulus",
        "Unnamed: 17": "300% Modulus",
        "Unnamed: 21": "Density, g/cm³"
    })

    gl_rows_copy = gl_rows_copy.drop(columns=['Unnamed: 8', '50% Modulus', '200% Modulus'])

    # Extract the Lot value
    file_name_with_extension = os.path.basename(file_path)
    file_name_without_extension = os.path.splitext(file_name_with_extension)[0]

    # Extract the Date value
    Date = df1.loc[2, "Unnamed: 10"]

    # Extract the QC value
    QC = df1.loc[1, "Unnamed: 10"]

    # Extract PO Number
    PO = df1.loc[3, "Unnamed: 5"]

    # Adding new columns
    gl_rows_copy.insert(0, 'Lot', file_name_without_extension)
    gl_rows_copy.insert(1, 'Date', Date)
    gl_rows_copy.insert(3, 'QC', QC)
    gl_rows_copy.insert(4, 'PO Number', PO)

    # Specify output path
    output_path = output_directory + file_name_without_extension + ' (cleaned).xlsx'

    # Convert 'Date' to datetime, if it's not already
    gl_rows_copy['Date'] = pd.to_datetime(gl_rows_copy['Date'])

    # Format 'Date' column in 'YYYY-MM-DD' format
    gl_rows_copy['Date'] = gl_rows_copy['Date'].dt.strftime('%Y-%m-%d')

    # Save the DataFrame to Excel
    gl_rows_copy.to_excel(output_path, index=False)


/var/folders/x0/hs0ftcf97kvfw04xwdy3zf_00000gn/T/ipykernel_33725/4083916118.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gl_rows['Box Number'] = gl_rows['Production Run Quality Record'].str.split(' ').str[1]
/var/folders/x0/hs0ftcf97kvfw04xwdy3zf_00000gn/T/ipykernel_33725/4083916118.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gl_rows['Box Number'] = gl_rows['Production Run Quality Record'].str.split(' ').str[1]
/var/folders/x0/hs0ftcf97kvfw04xwdy3zf_00000gn/T/ipykernel_33725/4083916118.py:2

In [6]:
from openpyxl import load_workbook
import re

Template = "/Users/lutianshu/Desktop/Polymax/P31-466-A Template.xlsx"

# specify the directory path
directory_path = "/Users/lutianshu/Desktop/Polymax/Clean/"

# get a list of all Excel files in the directory
excel_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.xlsx') or f.endswith('.xls')]

# Read them into pandas dataframes and combine them
combined_df = pd.concat([pd.read_excel(f) for f in excel_files])

# Remove all characters before the first digit in the 'Lot' column
combined_df['Lot'] = combined_df['Lot'].apply(lambda x: re.sub('^[^\d]*', '', str(x)))

# Add "%" to the 'Elongation' column, ignoring NaN values
combined_df['Elongation'] = combined_df['Elongation'].apply(lambda x: str(x) + "%" if pd.notnull(x) else x)

# Load the writer and the template file
book = load_workbook(Template)
writer = pd.ExcelWriter(Template, engine='openpyxl') 
writer.book = book

# Make sure that the header is not written again
startrow = writer.sheets['Sheet1'].max_row

# Write data to the sheet named 'Sheet1' 
combined_df.to_excel(writer, startrow=startrow, index=False, header=False)

# Save the file
writer.save()
writer.close()

/var/folders/x0/hs0ftcf97kvfw04xwdy3zf_00000gn/T/ipykernel_33725/2296263862.py:24: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/var/folders/x0/hs0ftcf97kvfw04xwdy3zf_00000gn/T/ipykernel_33725/2296263862.py:33: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
